In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf

Using TensorFlow backend.


# Data Preperation

In [16]:
np.random.seed(0)

data = pd.read_csv("./data/handwritten_data_785.csv", encoding = "utf8")
print(data.shape)

(372037, 785)


In [17]:
data = data.values
np.random.shuffle(data)
X, Y = data[:,1:], data[:,0]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [18]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

classes = np.unique(Y_train)
num_classes = len(classes)

X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)

In [19]:
mean_image = np.mean(X_train, axis=0).astype(np.int64)
X_train = (X_train - mean_image)/255
X_test = (X_test - mean_image)/255

# Testing Layers
## [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]

In [59]:

from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import LeakyReLU, ELU
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from keras.utils import to_categorical

from tensorflow.python.keras import initializers

optimizer = Adam(lr=1e-3)

In [69]:
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

model = Sequential()

#conv
#keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, 
    #dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
    #bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
    #activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
model.add(Conv2D(filters=10, kernel_size=5, strides = 1, padding='same', activation='linear',
                 bias_initializer='RandomNormal', input_shape=(img_size,img_size,1)))

#ReLu
model.add(ELU(alpha=1.0))

#conv2
model.add(Conv2D(filters=10, kernel_size=5, strides = 1, padding='same', activation='linear',
                 bias_initializer='RandomNormal', input_shape=(img_size,img_size,1)))

#ReLu
model.add(ELU(alpha=1.0))

#MaxPool
#keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)
model.add(MaxPooling2D(strides=1, padding='same'))

#Affine
model.add(Flatten())

#keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
    #bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
    #activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 28, 28, 10)        260       
_________________________________________________________________
elu_17 (ELU)                 (None, 28, 28, 10)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 28, 28, 10)        2510      
_________________________________________________________________
elu_18 (ELU)                 (None, 28, 28, 10)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 28, 28, 10)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 7840)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 26)                203866    
Total para

In [70]:
model.fit(x=X_train,
          y=train_Y_one_hot,
          epochs=3, batch_size=128,verbose=1, validation_data=(X_test, test_Y_one_hot))

Train on 297629 samples, validate on 74408 samples
Epoch 1/3
297629/297629 [==============================] - 640s 2ms/step - loss: 0.3055 - acc: 0.9191 - val_loss: 0.1483 - val_acc: 0.9613
Epoch 2/3
297629/297629 [==============================] - 619s 2ms/step - loss: 0.1395 - acc: 0.9632 - val_loss: 0.1340 - val_acc: 0.9653
Epoch 3/3
297629/297629 [==============================] - 1324s 4ms/step - loss: 0.1232 - acc: 0.9673 - val_loss: 0.1229 - val_acc: 0.9678
